In [1]:
from pymystem3 import Mystem
my_stem = Mystem()

import nltk

try:
    nltk.corpus.stopwords.words('english')
except:
    nltk.download('stopwords')

import pandas as pd
from collections import defaultdict

import gensim.models

DATA_PATH = '/data/'
print('success')

success


In [3]:
from nltk.corpus import stopwords
from gensim.parsing.preprocessing import strip_multiple_whitespaces
from gensim.parsing.preprocessing import strip_non_alphanum


useless_symbols = set(['_', 'ー'])
filter_words = set()
# load stopwords from nltk
filter_words = filter_words.union(set(stopwords.words('english')))
filter_words = filter_words.union(set(stopwords.words('russian')))
# load own stopwords
with open('stopwords_full.txt') as f:
    filter_words = filter_words.union(set([word for line in f for word in line.split()]))


# input: document: str
# output: document: str
# make preprocessing of documents, remove useless symbols, remove useless words
def preprocessor(s: str):
    # all letter to lower case
    s = s.lower() 
    
    # remove all symbol, which aren't letter or num
    s = strip_non_alphanum(s)
    # strip_non_alphanum don't remove '_' symbol
    s = ''.join(map(lambda c: ' ' if c in useless_symbols else c, s))

    s = ''.join(my_stem.lemmatize(s))
    # replace all space symbol like space, tab, \n to simple space and remove double and for space
    s = strip_multiple_whitespaces(s)
    # remove empty words and words from filtered words 
    s = ' '.join(list(filter(lambda w:  len(w) > 0 and not (w in filter_words or w.isdigit()), s.split(' '))))
    return s


# load file and drop documents with empty text
df = pd.read_csv(DATA_PATH + 'events.csv')
# preprocess of corpus
df['description'] = df['description'].apply(preprocessor)

# save results
df.to_csv(r'' + DATA_PATH + 'events_norm.csv', index=False)
print("success")

3979
3979
3979
3979
success


In [34]:
import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv(DATA_PATH + "events_norm.csv")
number_topics = 25
lda = LatentDirichletAllocation(n_components=number_topics, n_jobs=20)

documents = [None] * df.shape[0]
dictionary = {}

for index, row in df.iterrows():
    words = row.description.split()
    
    for word in words:
        value = dictionary[word] if word in dictionary else 0
        dictionary[word] = value + 1

for index, row in df.iterrows():
    documents[index] = ' '.join(filter(lambda word: dictionary[word] > 1, row.description.split()))
    

count_vect = TfidfVectorizer()
count_data = count_vect.fit_transform(documents)
lda.fit(corpus_count)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=25, n_jobs=20,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)

In [36]:
print("Topics found via LDA:")
import numpy as np


words = count_vect.get_feature_names()

for topic_idx, topic in enumerate(lda.components_):
    print("\nTopic #%d:" % topic_idx)
    print(" ".join([words[i]
                    for i in topic.argsort()[:-30 - 1:-1]]))


Topics found via LDA:

Topic #0:
ска хоккей hcska ботаническийсад ska hockey победа кхл ледовыйдворец entershikari матч динамо армейцыснев игра ледовый энергияпитер хкск khl ботанический толькоскаитолькопобеда цска hollywoodundead сакура болеть шайба йокерит icehockey сезон команда хоккейный

Topic #1:
9мая деньпобеда би2 notredamedeparis победа мариинскийтеатр мюзикл ladiestrophy tennis 9мая2019 теннис mariinsky ледовыйдворец formulatx b2band балет mariinskytheatre опера поранатеннис мариинка мариинский сднемпобеда театр мирный деньпобеды2019 кипел newbest би2newbest би ladiestrophy2019

Topic #2:
тюльпан фестивальтюльпан елагиностров цпкио елагин tulips цветок фестивальтюльпанов2019 flowers elaginisland elaginpark елагинпарк цпкиокиров елагиностровый фестивальтюльпановспб тюльпаномания tulip tulipfestival елагиностер остров елагиностр сорт белка elagin yelaginisland белок цвести maruv anacondaz тюльпановый

Topic #3:
twentyonepilots bulletformyvalentine banditotour bfmv кудрово joshu

In [35]:
from pyLDAvis import sklearn as sklearn_lda
import pyLDAvis


LDAvis_prepared = sklearn_lda.prepare(lda, count_data, count_vect)
pyLDAvis.display(LDAvis_prepared)